# Using Ungraph Library

Este notebook muestra cómo usar la librería Ungraph para convertir datos no estructurados en grafos de conocimiento.

## Pipeline Completo:
1. Configurar conexión a Neo4j
2. Obtener recomendación de chunking
3. Ingerir documentos al grafo
4. Buscar información en el grafo


In [1]:
def add_src_to_path(path_folder: str):
    ''' 
    Helper function for adding the "path_folder" directory to the path.
    in order to work on notebooks and scripts
    '''
    import sys
    from pathlib import Path

    base_path = Path().resolve()
    for parent in [base_path] + list(base_path.parents):
        candidate = parent / path_folder
        if candidate.exists():
            # Agregar el directorio padre para que sea un paquete Python
            parent_dir = candidate.parent
            if str(parent_dir) not in sys.path:
                sys.path.insert(0, str(parent_dir))
                print(f"Path Folder parent added: {parent_dir}")
            if str(candidate) not in sys.path:
                sys.path.append(str(candidate))
                print(f"Path Folder {path_folder} added: {candidate}")
            return
    print(f"Not found '{path_folder}' folder on the hierarchy of directories")

# Agregar carpetas necesarias al path
# Esto permite importar src como un paquete Python
add_src_to_path(path_folder="src")
add_src_to_path(path_folder="src/utils")
add_src_to_path(path_folder="src/data")


Path Folder parent added: D:\projects\Ungraph
Path Folder src added: D:\projects\Ungraph\src
Path Folder src/utils added: D:\projects\Ungraph\src\utils
Path Folder src/data added: D:\projects\Ungraph\src\data


In [2]:
# Importar la librería Ungraph y utilidades
import sys
from pathlib import Path

# Importar handlers desde utils (ya está en el path)
from src.utils.handlers import find_in_project

# Importar ungraph
# Intentar importar como paquete instalado primero
try:
    import ungraph
    print("✅ Ungraph importado como paquete instalado")
except ImportError:
    # Si no está instalado, importar desde src
    # src/__init__.py contiene toda la API pública
    import src
    ungraph = src
    print("✅ Ungraph importado desde src/ (modo desarrollo)")

print(f"📦 Ungraph version: {ungraph.__version__}")
print(f"🔧 Funciones disponibles: {[f for f in dir(ungraph) if not f.startswith('_')][:10]}")


d:\projects\Ungraph\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Ungraph importado desde src/ (modo desarrollo)
📦 Ungraph version: 0.1.0
🔧 Funciones disponibles: ['Any', 'Chunk', 'ChunkingMaster', 'ChunkingRecommendation', 'ChunkingResult', 'ChunkingStrategy', 'Dict', 'HuggingFaceEmbeddingService', 'IngestDocumentUseCase', 'LangChainDocument']


## 1. Configuración

Primero configuramos la conexión a Neo4j. Puedes usar variables de entorno o configuración programática.


In [3]:
# ⚠️ IMPORTANTE: Configura tus credenciales de Neo4j antes de continuar

# Opción 1: Usar variables de entorno (recomendado)
# En terminal:
# export NEO4J_URI="bolt://localhost:7687"
# export NEO4J_PASSWORD="tu_contraseña_real"

# Opción 2: Configurar programáticamente
# ⚠️ REEMPLAZA "tu_contraseña" con tu contraseña real de Neo4j
ungraph.configure(
    neo4j_uri="bolt://localhost:7687",
    neo4j_user="neo4j",
    neo4j_password="Ungraph22",  # ⚠️ CAMBIAR: Usa tu contraseña real de Neo4j
    neo4j_database="neo4j",
    embedding_model="sentence-transformers/all-MiniLM-L6-v2"
)

print("✅ Configuración completada")
print("💡 Si obtienes AuthError, verifica que:")
print("   1. Neo4j esté corriendo")
print("   2. Las credenciales sean correctas")
print("   3. La URI sea accesible")


✅ Configuración completada
💡 Si obtienes AuthError, verifica que:
   1. Neo4j esté corriendo
   2. Las credenciales sean correctas
   3. La URI sea accesible


## 2. Encontrar Archivos de Datos

Localizamos los archivos de prueba en la carpeta `data`.


In [4]:
# Encontrar la carpeta data de manera agnóstica
data_path = find_in_project(
    target="data",
    search_type="folder",
    project_root=None
)

print(f"📁 Carpeta de datos encontrada: {data_path}")

# Listar archivos disponibles
files = list(data_path.glob("*"))
print(f"\n📄 Archivos disponibles:")
for file in files:
    if file.is_file():
        print(f"  - {file.name} ({file.stat().st_size / 1024:.1f} KB)")


📁 Carpeta de datos encontrada: D:\projects\Ungraph\src\data

📄 Archivos disponibles:
  - 110225.md (6.6 KB)
  - AnnyLetter.txt (17.8 KB)
  - Usar símboles de silencio de corchea.docx (25.2 KB)


## 3. Obtener Recomendación de Chunking

Antes de ingerir, podemos obtener una recomendación sobre la mejor estrategia de chunking para cada documento.


In [5]:
# Obtener recomendación para un archivo Markdown
markdown_file = data_path / "110225.md"

if markdown_file.exists():
    print(f"📊 Analizando: {markdown_file.name}")
    recommendation = ungraph.suggest_chunking_strategy(markdown_file)
    
    print(f"\n✅ Recomendación obtenida:")
    print(f"   Estrategia: {recommendation.strategy}")
    print(f"   Chunk size: {recommendation.chunk_size}")
    print(f"   Chunk overlap: {recommendation.chunk_overlap}")
    print(f"   Quality score: {recommendation.quality_score:.2f}")
    print(f"\n📝 Explicación:")
    print(recommendation.explanation)
else:
    print(f"⚠️  Archivo no encontrado: {markdown_file}")


📊 Analizando: 110225.md

✅ Recomendación obtenida:
   Estrategia: recursive
   Chunk size: 2000
   Chunk overlap: 300
   Quality score: 70.80

📝 Explicación:
Se recomienda la estrategia 'recursive' porque:
- El documento es Markdown con estructura de headers
- El documento tiene 29 párrafos
- Generará aproximadamente 4 chunks
- Tamaño promedio de chunk: 1792 caracteres
- Score de calidad: 70.80/1.0


## 4. Ingerir Documentos al Grafo

Ahora ingerimos los documentos usando la recomendación o parámetros personalizados.


In [6]:
# Ingerir archivo Markdown
markdown_file = data_path / "110225.md"

if markdown_file.exists():
    print(f"📥 Ingiriendo: {markdown_file.name}")
    chunks = ungraph.ingest_document(
        markdown_file,
        chunk_size=1000,
        chunk_overlap=200,
        clean_text=True
    )
    print(f"✅ Documento ingerido exitosamente!")
    print(f"   Total de chunks: {len(chunks)}")
    print(f"   Chunks con embeddings: {sum(1 for c in chunks if c.embeddings)}")
    
    # Mostrar información de algunos chunks
    print(f"\n📄 Primeros 3 chunks:")
    for i, chunk in enumerate(chunks[:3], 1):
        print(f"\n   Chunk {i}:")
        print(f"   - ID: {chunk.id[:50]}...")
        print(f"   - Contenido: {chunk.page_content[:100]}...")
        print(f"   - Consecutivo: {chunk.chunk_id_consecutive}")
        if chunk.embeddings:
            print(f"   - Embeddings: {len(chunk.embeddings)} dimensiones")
else:
    print(f"⚠️  Archivo no encontrado: {markdown_file}")


📥 Ingiriendo: 110225.md
Chunk relationships created successfully
✅ Documento ingerido exitosamente!
   Total de chunks: 9
   Chunks con embeddings: 9

📄 Primeros 3 chunks:

   Chunk 1:
   - ID: 110225.md_e6fd4c94-6635-4704-b161-c9dbdae794d4...
   - Contenido: Incluso es lo primero que se abrio, quiza lo que me falta en este momento es la palabra, y necesite ...
   - Consecutivo: 1
   - Embeddings: 384 dimensiones

   Chunk 2:
   - ID: 110225.md_5ff5f5a6-4331-4084-bd21-cb111f92a40c...
   - Contenido: quiero narrar como estoy desarrollando la vision de mi mismo, la que quiero para mi, el humano que q...
   - Consecutivo: 2
   - Embeddings: 384 dimensiones

   Chunk 3:
   - ID: 110225.md_9bf3da65-3618-48fd-94c6-369b7223ba08...
   - Contenido: cobran sentido en la contradiccion de su naturaleza. Quiza lo que me falta es el ejercicio diario de...
   - Consecutivo: 3
   - Embeddings: 384 dimensiones


In [7]:
# Ingerir archivo de texto
txt_file = data_path / "AnnyLetter.txt"

if txt_file.exists():
    print(f"📥 Ingiriendo: {txt_file.name}")
    chunks_txt = ungraph.ingest_document(
        txt_file,
        chunk_size=500,
        chunk_overlap=100
    )
    print(f"✅ Documento ingerido exitosamente!")
    print(f"   Total de chunks: {len(chunks_txt)}")
else:
    print(f"⚠️  Archivo no encontrado: {txt_file}")


📥 Ingiriendo: AnnyLetter.txt
Chunk relationships created successfully
✅ Documento ingerido exitosamente!
   Total de chunks: 45


In [8]:
# Ingerir archivo Word
docx_file = data_path / "Usar símboles de silencio de corchea.docx"

if docx_file.exists():
    print(f"📥 Ingiriendo: {docx_file.name}")
    chunks_docx = ungraph.ingest_document(
        docx_file,
        chunk_size=1000,
        chunk_overlap=200
    )
    print(f"✅ Documento ingerido exitosamente!")
    print(f"   Total de chunks: {len(chunks_docx)}")
else:
    print(f"⚠️  Archivo no encontrado: {docx_file}")


📥 Ingiriendo: Usar símboles de silencio de corchea.docx
Chunk relationships created successfully
✅ Documento ingerido exitosamente!
   Total de chunks: 27


## 5. Buscar en el Grafo

Ahora podemos buscar información en el grafo usando diferentes métodos de búsqueda.


In [9]:
# Búsqueda simple por texto
query = "computación cuántica"
print(f"🔍 Buscando: '{query}'")

results = ungraph.search(query, limit=5)

if results:
    print(f"\n✅ Encontrados {len(results)} resultados:\n")
    for i, result in enumerate(results, 1):
        print(f"Resultado {i}:")
        print(f"  Score: {result.score:.4f}")
        print(f"  Chunk ID: {result.chunk_id}")
        print(f"  Contenido: {result.content[:200]}...")
        print()
else:
    print("⚠️  No se encontraron resultados")
    print("💡 Asegúrate de haber ingerido documentos primero")


🔍 Buscando: 'computación cuántica'

✅ Encontrados 4 resultados:

Resultado 1:
  Score: 3.1815
  Chunk ID: 110225.md_2386e056-324b-470a-97f6-0b5a18a31d34
  Contenido: quiero narrar como estoy desarrollando la vision de mi mismo, la que quiero para mi, el humano que quiero moldear en este presente, y suena ambicioso, pero no me he rendido con la busqueda, no he tira...

Resultado 2:
  Score: 3.1815
  Chunk ID: 110225.md_5ff5f5a6-4331-4084-bd21-cb111f92a40c
  Contenido: quiero narrar como estoy desarrollando la vision de mi mismo, la que quiero para mi, el humano que quiero moldear en este presente, y suena ambicioso, pero no me he rendido con la busqueda, no he tira...

Resultado 3:
  Score: 3.1815
  Chunk ID: 110225.md_9748a95f-b378-4b91-85de-6cdd11c55ad1
  Contenido: quiero narrar como estoy desarrollando la vision de mi mismo, la que quiero para mi, el humano que quiero moldear en este presente, y suena ambicioso, pero no me he rendido con la busqueda, no he tira...

Resultado 4:
  Sc

In [10]:
# Búsqueda híbrida (texto + vectorial)
query = "inteligencia artificial"
print(f"🔍 Búsqueda híbrida: '{query}'")

results_hybrid = ungraph.hybrid_search(
    query,
    limit=3,
    weights=(0.3, 0.7)  # 30% texto, 70% vectorial
)

if results_hybrid:
    print(f"\n✅ Encontrados {len(results_hybrid)} resultados:\n")
    for i, result in enumerate(results_hybrid, 1):
        print(f"{'=' * 80}")
        print(f"Resultado {i}:")
        print(f"  Score combinado: {result.score:.4f}")
        print(f"  Chunk ID: {result.chunk_id}")
        
        if result.previous_chunk_content:
            print(f"\n  [Contexto Anterior]")
            print(f"  {result.previous_chunk_content[:150]}...")
        
        print(f"\n  [Contenido Principal]")
        print(f"  {result.content[:300]}...")
        
        if result.next_chunk_content:
            print(f"\n  [Contexto Siguiente]")
            print(f"  {result.next_chunk_content[:150]}...")
        
        print()
else:
    print("⚠️  No se encontraron resultados")


🔍 Búsqueda híbrida: 'inteligencia artificial'


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (text_node) { ... }} {position: line: 7, column: 9, offset: 215} for query: '\n        // Búsqueda fulltext\n        CALL db.index.fulltext.queryNodes("chunk_content", $query_text)\n        YIELD node as text_node, score as text_score\n        \n        // Combinar con búsqueda vectorial\n        CALL {\n            WITH text_node\n            CALL db.index.vector.queryNodes(\'chunk_embeddings\', toInteger($top_k), $query_vector)\n            YIELD node as vec_node, score as vec_score\n            WHERE text_node = vec_node\n            RETURN vec_node, vec_score\n        }\n        \n        // Calcular score combinado\n        WITH text_node as node, text_score, vec_score,\n


✅ Encontrados 3 resultados:

Resultado 1:
  Score combinado: 1.2027
  Chunk ID: 110225.md_2386e056-324b-470a-97f6-0b5a18a31d34

  [Contexto Anterior]
  Incluso es lo primero que se abrio, quiza lo que me falta en este momento es la palabra, y necesite encontrarme en ella para retomar mi camino, hoy mi...

  [Contenido Principal]
  quiero narrar como estoy desarrollando la vision de mi mismo, la que quiero para mi, el humano que quiero moldear en este presente, y suena ambicioso, pero no me he rendido con la busqueda, no he tirado la toalla a falta de mejores opciones, no me resigno. Si tuviera algo hecho en 5 anos, seria nues...

  [Contexto Siguiente]
  cobran sentido en la contradiccion de su naturaleza. Quiza lo que me falta es el ejercicio diario de escribir, de narrarme el hombre que deje de ser, ...

Resultado 2:
  Score combinado: 1.2027
  Chunk ID: 110225.md_2386e056-324b-470a-97f6-0b5a18a31d34

  [Contexto Anterior]
  Incluso es lo primero que se abrio, quiza lo que me falta 

## 6. Pipeline Completo con Múltiples Archivos

Ejemplo de cómo procesar múltiples archivos en un pipeline completo.


In [11]:
# Pipeline completo para múltiples archivos
files_to_process = [
    data_path / "110225.md",
    data_path / "AnnyLetter.txt",
    data_path / "Usar símboles de silencio de corchea.docx"
]

print("🚀 Iniciando pipeline completo...\n")

total_chunks = 0
for file_path in files_to_process:
    if file_path.exists():
        print(f"📄 Procesando: {file_path.name}")
        
        # 1. Obtener recomendación
        try:
            recommendation = ungraph.suggest_chunking_strategy(file_path)
            print(f"   💡 Estrategia recomendada: {recommendation.strategy}")
        except Exception as e:
            print(f"   ⚠️  No se pudo obtener recomendación: {e}")
            recommendation = None
        
        # 2. Ingerir documento
        try:
            if recommendation:
                chunks = ungraph.ingest_document(
                    file_path,
                    chunk_size=recommendation.chunk_size,
                    chunk_overlap=recommendation.chunk_overlap
                )
            else:
                chunks = ungraph.ingest_document(file_path)
            
            total_chunks += len(chunks)
            print(f"   ✅ {len(chunks)} chunks creados\n")
        except Exception as e:
            print(f"   ❌ Error al ingerir: {e}\n")

print(f"🎉 Pipeline completado!")
print(f"   Total de chunks procesados: {total_chunks}")


🚀 Iniciando pipeline completo...

📄 Procesando: 110225.md
   💡 Estrategia recomendada: recursive
Chunk relationships created successfully
   ✅ 4 chunks creados

📄 Procesando: AnnyLetter.txt
   💡 Estrategia recomendada: character
Chunk relationships created successfully
   ✅ 11 chunks creados

📄 Procesando: Usar símboles de silencio de corchea.docx
   💡 Estrategia recomendada: recursive
Chunk relationships created successfully
   ✅ 13 chunks creados

🎉 Pipeline completado!
   Total de chunks procesados: 28


## 7. Ejemplo de Búsqueda con Contexto Completo

Mostrar cómo construir contexto completo usando los resultados de búsqueda.


In [12]:
# Búsqueda con construcción de contexto completo
query = "Alejandro"
print(f"🔍 Buscando: '{query}'\n")

results = ungraph.hybrid_search(query, limit=2)

for i, result in enumerate(results, 1):
    # Construir contexto completo
    contexto_completo = ""
    
    if result.previous_chunk_content:
        contexto_completo += f"[CONTEXTO ANTERIOR]\n{result.previous_chunk_content}\n\n"
    
    contexto_completo += f"[CONTENIDO PRINCIPAL]\n{result.content}\n\n"
    
    if result.next_chunk_content:
        contexto_completo += f"[CONTEXTO SIGUIENTE]\n{result.next_chunk_content}"
    
    print(f"{'=' * 80}")
    print(f"Resultado {i} (Score: {result.score:.4f})")
    print(f"{'=' * 80}")
    print(contexto_completo)
    print()


🔍 Buscando: 'Alejandro'



Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (text_node) { ... }} {position: line: 7, column: 9, offset: 215} for query: '\n        // Búsqueda fulltext\n        CALL db.index.fulltext.queryNodes("chunk_content", $query_text)\n        YIELD node as text_node, score as text_score\n        \n        // Combinar con búsqueda vectorial\n        CALL {\n            WITH text_node\n            CALL db.index.vector.queryNodes(\'chunk_embeddings\', toInteger($top_k), $query_vector)\n            YIELD node as vec_node, score as vec_score\n            WHERE text_node = vec_node\n            RETURN vec_node, vec_score\n        }\n        \n        // Calcular score combinado\n        WITH text_node as node, text_score, vec_score,\n

Resultado 1 (Score: 0.8439)
[CONTEXTO ANTERIOR]
y testeo de aplicaciones desarrolladas fullstack con AI para ser mas competente a nivel profesional. Ser mejor en quantum computing para poder sembrar buenas bases y raices de lo que quiero construir para mi futuro profesional. Aprender sobre Quimica Computacional para desempenarme realmente con lo qu estamos construyendo. Aprovechar sobre la complejidad, los grafos de conocimiento y la ingeniera de datos con Neo4j y analitica de grafos para poder mejorar mis skills como ingeniero de la complejidad. Dado que requiero desempeno mental muy alto y nitido necesito tener claro que las acciones para lograr esto derivan en: Mente no intoxicada, Bien alimentada, con un cuerpo que se mueve y le genera lso estimulos suficientes para crear coenxiones, descanso y reposo necesarios, sesiones de trabajo enfocado, de descanso real, remover el ocio de series, porno , fantasias del conducto de agua caliente, ([[Teoria del Conducto del Agua Caliente]]), si

## Resumen

Este notebook muestra cómo usar Ungraph para:

1. ✅ Configurar conexión a Neo4j (variables de entorno o programática)
2. ✅ Obtener recomendaciones de chunking con explicaciones
3. ✅ Ingerir documentos (Markdown, TXT, Word)
4. ✅ Buscar información (texto simple y búsqueda híbrida)
5. ✅ Construir contexto completo para RAG

### Próximos Pasos

- Explorar más opciones de configuración
- Probar con diferentes modelos de embeddings
- Experimentar con diferentes estrategias de chunking
- Crear patrones personalizados de estructura en el grafo
